In [39]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [43]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", 
                   input_shape=IMAGE_SHAPE+(3,))
])

classifier.include_top = False

In [44]:
classifier.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 1001)              3540265   
Total params: 3,540,265
Trainable params: 0
Non-trainable params: 3,540,265
_________________________________________________________________


In [83]:
def list_directories(path):
    """list files and directories in a given path"""
    arr = os.listdir(path)
    return arr

In [87]:

def cnn_feature_extraction(image):
    
    image = np.array(image)/255.0
    
    pre = classifier.predict(image[np.newaxis])
    pre = np.array(pre)
    query_feature = pre.flatten()
    
    return query_feature
    


In [88]:
def calculate_features():
    
    dicte = {}

    images = []
    file = 'images/'
    indexies_path = 'indexies/'
    imgs = list_directories(file)
    
    for i in imgs:
        
        pth = f"{file}{i}"
        images.append(pth)
        
    for imagePath, imageID in zip(images, imgs):
        
#         image = cv2.imread(imagePath)
        image = Image.open(imagePath).resize(IMAGE_SHAPE)

        features = cnn_feature_extraction(image)
        dicte[imageID] = features.tolist()
        
    with open(f'{indexies_path}cnn_indexies.json', 'w') as f:
        json.dump(dicte, f)
        


In [102]:
def search_cnn(query_image='gf.jpg'):
    
    best_matches = []
    temp_matches = {}
    
    
    file = 'images/'
    indexies_path = 'indexies/'
    
#     query = cv2.imread('images/gf.jpg')
#     query = cv2.imread(f'images/{query_image}')

    query = Image.open(f'images/{query_image}').resize(IMAGE_SHAPE)

    query_features = np.array(cnn_feature_extraction(query),np.float32)
    
    
    f = open (f'{indexies_path}cnn_indexies.json', "r")
    data = json.loads(f.read())
    
    for ID,f in data.items():
        
        indexed_features = np.array(f,np.float32)
        
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        matches = bf.knnMatch(query_features,indexed_features, k=2)
        
        good = []
        for m,n in matches:  ## read indivusal distance for each detected match between 2 images
            if m.distance < 0.75*n.distance:
                good.append([m])
        temp_matches[ID] = len(good)
        
    sorted_temp_matches = sorted(temp_matches.items(), key=lambda kv: kv[1])
#     print(sorted_temp_matches)
    print(sorted_temp_matches[-1])

search_cnn()

('gf.jpg', 1001)


In [98]:
calculate_features()